# Explore: Amtrak Network

## Intercity Passenger Rail Service Station Performance Metrics

The Amtrak [network](https://www.amtrak.com/content/dam/projects/dotcom/english/public/documents/Maps/Amtrak-System-Map-020923.pdf)
is a passenger rail service that provides intercity rail service in the
continental United States and to select Canadian cities. The network is operated by the
[National Railroad Passenger Corporation](https://railroads.dot.gov/passenger-rail/amtrak/amtrak),
a federally chartered for-profit corporation that receives some state funding and covers its
operating costs by selling tickets and providing other services.

This notebook commences exploration of the augmented quarterly
[Amtrak](https://www.amtrak.com/home.html) station performance metrics. The goal is to better
understand the Amtrak network as a whole and identify potential areas for further analysis.

### Variable names

A number of variable names in this project leverage the following abbreviations. The naming
strategy is to strike a balance between brevity and readability:

* `amtk`: Amtrak (reporting mark)
* `chrt`: chart
* `cols`: columns
* `const`: constant
* `cwd`: current working directory
* `eb`: eastbound direction of travel
* `lm`: linear model
* `mi`: miles
* `mm`: minutes (ISO 8601)
* `nb`: northbound direction of travel
* `psgr`: passenger
* `qtr`: quarter
* `rte`: route
* `sb`: southbound direction of travel
* `stats`: summary statistics
* `stn`: station
* `stns`: stations
* `svc`: service
* `trn`: train
* `wb`: westbound direction of travel

In [ ]:
import numpy as np
import pandas as pd
import pathlib as pl
import scipy.stats as stats
import tomllib as tl

import fra_amtrak.amtk_detrain as detrn
import fra_amtrak.amtk_frame as frm
import fra_amtrak.chart_bar as bar
import fra_amtrak.chart_box_preagg as boxp
import fra_amtrak.chart_hist as hst
import fra_amtrak.chart_title as ttl

## 1.0 Read files

### 1.1 Resolve paths


In [ ]:
parent_path = pl.Path.cwd()  # current working directory
parent_path

### 1.2 Load constants

Load a companion [TOML](https://toml.io/en/) file named `notebook.toml` containing constants.

In [ ]:
filepath = parent_path.joinpath("notebook.toml")
with open(filepath, "rb") as file_obj:
    const = tl.load(file_obj)

# Access constants
AGG = const["agg"]
CHRT_BAR = const["chart"]["bar"]
COLORS = const["colors"]
COLS = const["columns"]


### 1.3 Retrieve performance data

In [ ]:
filepath = parent_path.joinpath("data", "station_performance_metrics-v1p2.csv")
network = pd.read_csv(
    filepath, dtype={"Address 02": "str", "ZIP Code": "str"}, low_memory=False
)  # avoid DtypeWarning


### 1.4 Review the `DataFrame`

In [ ]:
network.shape

In [ ]:
network.info()

In [ ]:
network.head(3)

## 2.0 The Amtrak network

Network-wide summary statistics covering all available fiscal years and quarters. The data is
derived by flattening a `DataFrame` into a single row of metrics.

In [ ]:
# Total train arrivals
network_trn_arrivals = network.shape[0]

# Detraining totals
network_detrn = network[COLS["total_detrn"]].sum()
network_detrn_late = network[COLS["late_detrn"]].sum()
network_detrn_on_time = network_detrn - network_detrn_late

print(
    f"Train Arrivals: {network_trn_arrivals}",
    f"Total Detraining Customers: {network_detrn}",
    f"Late Detraining Customers: {network_detrn_late}",
    f"On-Time Detraining Customers: {network_detrn_on_time}",
    sep="\n",
)

# Compute summary statistics
network_stats = detrn.get_sum_stats(network, AGG["columns"], AGG["funcs"])
network_stats

### 2.1 Service lines

The Amtrak network consists of several service lines. The service lines are used to group
services, stations, and train routes. Service line summary statistics cover all available fiscal
years and quarters and are derived by aggregating metrics across each service line.

In [ ]:
svc_line_stats = detrn.get_sum_stats_by_group(
    network, COLS["svc_line"], AGG["columns"], AGG["funcs"]
)
svc_line_stats

### 2.2 Services

Each Amtrak service line consists of one or more services. A service is a named train (e.g., the
[_California Zephyr_](https://www.amtrak.com/california-zephyr-train)) that operates on a specific
route.

Note that certain state-supported services
(e.g., [Michigan service](https://www.amtrak.com/michigan-services-train)) represent a roll up of
one or more named train sub services. The named trains that comprise these services are categorized
as sub services and are excluded from the services column.

In [ ]:
serv = network.loc[:, COLS["svc"]].unique()
serv.sort()

print(f"services (n={np.count_nonzero(serv)})")
serv

Service-level summary statistics cover all available fiscal years and quarters and are derived by
aggregating metrics across each service.

In [ ]:
svc_stats = detrn.get_sum_stats_by_group(network, COLS["svc"], AGG["columns"], AGG["funcs"])
svc_stats

### 2.3 Sub services

The Amtrak sub service category is used to group named trains that operate as part of a service such
as the [Illinois](https://www.amtrak.com/illinois-services-train) or
[Michigan](https://www.amtrak.com/michigan-services-train) services (e.g., _Blue Water_,
_Carl Sandburg_, _Illinois Zephyr_, _Pere Marquette_, _Saluki_, and _Wolverine_). Note that
service-level named trains (e.g., _Acela_) are also included in the sub service category.

In [ ]:
sub_serv = network.loc[:, COLS["sub_svc"]].unique()
sub_serv.sort()

print(f"subservices (n={np.count_nonzero(sub_serv)})")
sub_serv

Sub service-level summary statistics cover all available fiscal years and quarters and are derived
by aggregating metrics across each sub service.

In [ ]:
sub_svc_stats = detrn.get_sum_stats_by_group(network, COLS["sub_svc"], AGG["columns"], AGG["funcs"])
sub_svc_stats

### 2.4 Stations

Each `network` row represents Amtrak station arrival metrics, aggregated quarterly, for a given
train (e.g., _Wolverine_ Train 350 arrivals at [Ann Arbor](https://www.amtrak.com/stations/arb)
station (ARB) during Q3 2024).

Station-level summary statistics will be explored in a separate notebook.

In [ ]:
stn_count = network.loc[:, COLS["station_code"]].nunique()
stn_count

### 2.5 Regions and divisions

The Amtrak stations in `network` can be grouped by region and/or division. These geographical
groupings are based on US Census Bureau
[categories](https://www.census.gov/programs-surveys/economic-census/guidance-geographies/levels.html)
rather than FRA or Amtrak designations.

Group by stations by region. Ensure that regional station counts are consistent with the total station count.

#### 2.5.1 Regions [1 pt]

 The `DataFrame` named `region_stn_counts` provides stations counts grouped by region.

In [ ]:
region_stn_counts = (
    network.groupby(COLS["region"])[COLS["station_code"]]
    .nunique()
    .reset_index()
    .sort_values(by=COLS["region"])
)
region_stn_counts

In [ ]:
#hidden tests are within this cell

#### 2.5.2 Divisions [1 pt]

The `DataFrame` named `division_stn_counts` provides station counts grouped by region and divison.

In [ ]:
div_stn_counts = (
    network.groupby([COLS["region"], COLS["division"]])[COLS["station_code"]]
    .nunique()
    .reset_index()
    .sort_values(by=[COLS["region"], COLS["division"]])
)
div_stn_counts

In [ ]:
#hidden tests are within this cell

## 3.0 On-time performance metrics (entire period)

Amtrak station performance data is a compilation of quarterly metrics that focus on late
detraining passengers. Detraining assengers are considered on-time if they arrive at their
destination no later than fifteen (`15`) minutes after their scheduled arrival time. All other
detraining passengers are considered late.

### 3.1 Mean late arrival times summary statistics [1 pt]

Review the central tendency, dispersion, and shape of mean late arrival times across the
network. Calling the custom function named `frm.describe_numeric_column()` will return an
"enriched" dictionary of summary statistics.

In [ ]:
# Drop missing values
network_avg_mm_late = network[COLS["late_detrn_avg_mm_late"]].dropna().reset_index(drop=True)

# Call the function
network_avg_mm_late_describe = None
network_avg_mm_late_describe

In [ ]:
#hidden tests are within this cell

The skewness and kurtosis values returned suggest that the distribution of mean late arrival times
of Amtrak trains are positively skewed and features a sharper peak and heavier right tail than a
normal distribution. Let's confirm this visually by generating a histogram.

### 3.2 Visualize distribution of mean late arrival times

Visualize mean late arrival times for the entire period. The data is binned prior to plotting.

#### 3.2.1 Create the chart data

In [ ]:
# Convert to DataFrame
network_avg_mm_late = network_avg_mm_late.to_frame(name=COLS["avg_mm_late"])

# Get mean and standard deviation
mu = network_avg_mm_late_describe["center"]["mean"]
sigma = network_avg_mm_late_describe["spread"]["std"]

# Get max value (for x-axis ticks); pad max value for chart display
max_val = network_avg_mm_late_describe["position"]["max"].astype(int)
max_val_ceil = (np.ceil(max_val / 10) * 10).astype(int)

# Create bins
network_avg_mm_late, bins, num_bins, bin_width = frm.create_bins(
    network_avg_mm_late, COLS["avg_mm_late"], 15
)

# Bin the data
chrt_data = frm.bin_data(network_avg_mm_late, COLS["avg_mm_late"], bins)
# chart_data

#### 3.2.2 Generate the histogram

In [ ]:
# Chart title
title_txt = "Amtrak Network Late Detraining Passengers"
title = ttl.format_title(network_stats, title_txt)

# Tooltips
tooltip_config = [
    {"shorthand": "bin_center:Q", "title": "Average Minutes Late", "format": None},
    {"shorthand": "count:Q", "title": "Late Arrivals Count", "format": None},
]

# Create and display the histogram
chart = hst.create_histogram(
    frame=chrt_data,
    x_shorthand="bin_center:Q",
    x_title="Average Minutes Late",
    y_shorthand="count:Q",
    y_title="Late Arrivals Count",
    y_stack=False,
    line_shorthand="Avg Min Late:Q",
    mu=mu,
    sigma=sigma,
    num_bins=num_bins,
    bin_width=bin_width,
    x_tick_count_max=max_val_ceil,
    bar_color=COLORS["amtk_blue"],
    mu_color=COLORS["amtk_red"],
    sigma_color=COLORS["anth_gray"],
    tooltip_config=tooltip_config,
    title=title,
    height=300,
    width=680,
)
chart.display()

## 4.0 On-time performance metrics (by fiscal year and quarter)

Compute OTP summary statistics per fiscal year and quarter. Add quarterly train arrival metrics to
the `DataFrame` named `network_qtr_stats`.

In [ ]:
# Get quarterly stats
network_qtr_stats = detrn.get_sum_stats_by_group(
    network,
    [COLS["year"], COLS["quarter"]],
    AGG["columns"],
    AGG["funcs"],
    network_trn_arrivals,
    network_detrn,
)
network_qtr_stats

### 4.1 Write to file [1 pt]

Write `network_qtr_stats` to a CSV file named `stu-amtk-network_qtr_stats.csv`. Store the file in the
`data/student` directory. Then compare it to the accompanying `fxt-amtk-network_qtr_stats.csv` file.
It must match line for line, character for character.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

## 5.0 Visualize detraining passengers (by fiscal year and quarter)

Visualize Amtrak's detraining passengers, both on-time and late, across all years and quarters with
a bar chart.

In [ ]:
# Assemble the data for the chart
chrt_data = bar.create_detrain_chart_frame(network_qtr_stats, CHRT_BAR["columns"])

# Create chart title
title_text = f"Amtrak {const['service_lines']['nec']} (NEC) Detraining Passengers"
title = ttl.format_title(network_stats, title_text)

# Grouped bar chart
chart = bar.create_grouped_bar_chart(
    chrt_data,
    "Fiscal Period:N",
    "Passengers:Q",
    "Arrival Status:N",
    CHRT_BAR["xoffset_sort"],
    CHRT_BAR["colors"],
    title,
)

chart.display()

## 6.0 Visualize distribution of mean late arrival times (by fiscal year and quarter)

Visualizing mean late arrival times grouped by fiscal year and quarter may reveal interesting
patterns.

The data is flattened prior to creating a series of box plots. The fiscal year and quarter column
values are combined (e.g., `< year >Q< quarter >`) by applying the function named
`detrn.format_year_quarter()` to each row to create a new column named "Fiscal Year Quarter". A
second column is also added to color code each quarter and its associated box plot.

### 6.1 Create the chart data [1 pt]

In [ ]:
cols = [COLS["year"], COLS["quarter"], COLS["late_detrn_avg_mm_late"]]

# Group by fiscal year and quarter and flatten
chrt_data = network.groupby(cols[:2])[cols].apply(lambda x: x).reset_index(drop=True)

# Add 'Fiscal Year Quarter' column


# Drop columns and reorder
chrt_data.drop([COLS["year"], COLS["quarter"]], axis=1, inplace=True)
chrt_data.insert(0, COLS["year_quarter"], chrt_data.pop(COLS["year_quarter"]))

# Add color column
colors = [COLORS["amtk_blue"], COLORS["amtk_red"]]
chrt_data.loc[:, "Color"] = chrt_data[COLS["year_quarter"]].apply(detrn.assign_color, colors=colors)
chrt_data.head()

In [ ]:
#hidden tests are within this cell

### 6.2 Preaggregate the chart data

Attempting to instantiate an instance of a Vega-Altair [`alt.Chart()`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.Chart.html) class by passing to it a dataset comprising more than `5000` rows will trigger a `MaxRowsError`. You can disable the `MaxRows` check by calling `alt.data_transformers.disable_max_rows()` method. However, disabling the check may result in performance issues, including browser crashes.

The preferred approach when [working with large datasets](https://altair-viz.github.io/user_guide/large_datasets.html#large-datasets) is to _preaggregate_ the data before generating a plot. This can be achieved "manually"&mdash;the approach adopted in this notebook&mdash;or by [installing](https://altair-viz.github.io/user_guide/large_datasets.html#installing-vegafusion[) and [enabling](https://altair-viz.github.io/user_guide/large_datasets.html#enabling-the-vegafusion-data-transformer) Altair's companion [vegafusion](https://vegafusion.io/) data transformer package.

In [ ]:
# Compute aggregation statistics
cols = [COLS["year_quarter"], COLS["late_detrn_avg_mm_late"]]

# Pre-aggregate the data
chrt_data = frm.aggregate_data(chrt_data, cols)

### 6.3 Generate the box plots

In [ ]:
# Create chart title
title_text = "Amtrak Network Late Detraining Passengers"
title = ttl.format_title(network_stats, title_text)

chart_horizontal = boxp.create_boxplot(
    data=chrt_data,
    x_shorthand="Late Detraining Customers Avg Min Late:Q",
    x_title="Average Minutes Late",
    y_shorthand="Fiscal Year Quarter:N",
    y_title="Period",
    box_size=20,
    outlier_shorthand="outliers:Q",
    color_shorthand="Color:N",
    chart_title=title,
    orient=boxp.Orient.HORIZONTAL,
    height=400,
    width=680,
)
chart_horizontal.display()

## 7.0 Distance traveled and late detraining passengers

A number of factors influence the likelihood of late detraining passengers. Distance traveled is
possibly one such factor. Does a linear relationship exist between a train's route miles and the
late arrival times experienced by late detraining passengers?

### 7.1 Data preparation

The first step involves preparing the data for the regression analysis.

In [ ]:
lm_data = network[[COLS["route_miles"], COLS["late_detrn_avg_mm_late"]]].reset_index(drop=True)
lm_data.shape

### 7.2 Linear regression [1 pt]

The code below calculates a linear least-squares regression for two sets of measurements. The
dependent variable is "Late Detraining Customers Avg Min Late". The independent variable is
"Route Miles". The `result` object contains the regression attributes.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

The linear regression analysis suggests that there is a moderate positive relationship between a train's route miles and mean late arrival times for late detraining passengers. The slope suggests that with every additional route mile traveled, the average minutes late for late detraining passengers increases by approximately `0.0338` minutes. The *R*² value indicates that around `25.5%` of the variability in the average minutes late can be explained by route miles, with the remaining variability due to other factors or random noise. The very small standard errors for both the slope and intercept coefficients suggest a high level of precision in these estimates.

### 7.3 Predictions

#### 7.3.1 Twenty-five mile intervals [1 pt]

Create a new `DataFrame` named `route_mi_intervals` comprising a single column named "Route Miles"
with values ranging from `0` to `2600` in increments of `25`.

Then apply the function `detrn.predict_avg_min_late()` to each of the "Route Miles" values to
generate predicted average late times for every twenty-five (`25`) miles of rail travel up to `2600`
miles.

For the starting zero (`0`) route mile mark, assign the predicted average late time to `0.0`
minutes. Round each predicted value to two decimal places. Assign each predicted value to a new
column named "Predicted Avg Min Late".

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

#### 7.3.2 Named train route miles

Create a `DataFrame` of predicted late times for named trains to combine with `route_mi_intervals`.
Retrieve each named train (e.g. the sub service) and their associate route miles from `network` and
store in two columns named "Sub Service" and "Route Miles". Assign the new `DataFrame` to a variable
named `trn_route_mi`.

Next, generate predictions for each row in `trn_route_mi`. Round the predictions to the second
(`2nd`) decimal place. Assign the predictions to a new column named "Predicted Avg Min Late".

Note that these predicted average late times are relevant only for late detraining passengers who
travel the entire route.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

### 7.4 Combine the data [1 pt]

Combine `route_mi_intervals` and `trn_route_mi`. Assign the new `DataFrame` to a variable named
`lm_predict`. Then sort the `DataFrame` rows by the route miles (ascending) and the sub service
(descending). Finally, reset the index.

In [ ]:
# Columns in play
cols = [COLS["route_miles"], COLS["predict_avg_mm_late"], COLS["sub_svc"]]

# Concatenate DataFrames, sort, and reset the index
lm_predict = pd.concat([route_mi_intervals, trn_route_mi], ignore_index=True)
lm_predict.sort_values(by=[cols[0], cols[-1]], ascending=[False, True], inplace=True)
lm_predict.reset_index(drop=True, inplace=True)
lm_predict

In [ ]:
#hidden tests are within this cell

### 7.5 Write to file [1 pt]

Write `lm_predict` to a CSV file named `stu-amtk-avg_min_late_predict.csv`. Store the file in the
`data/student` directory. Then compare it to the accompanying `fxt-amtk-avg_min_late_predict.csv` file.
It must match line for line, character for character.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

## 8.0 Watermark

In [ ]:
%load_ext watermark
%watermark -h -i -iv -m -v